# Handling netCDF files for simple climate models

Use test data and MarbleCMIP5Cube to show how this all works.

- load cube from file
- load cube using inbuilt
- acting on the cube
- plotting
- cutting the cube
- plotting timeseries of cuts
- plotting spatial plots of cuts

In [18]:
from os.path import join
import warnings

import numpy as np
import iris

from netcdf_scm.iris_cube_wrappers import SCMCube, MarbleCMIP5Cube

In [19]:
DATA_PATH_TEST = join("..", "tests", "test_data")
DATA_PATH_TEST_MARBLE_CMIP5_ROOT = join(DATA_PATH_TEST, "marble_cmip5", "cmip5")

## Loading a cube

### Loading with iris

Here we show how to load a cube directly using iris.

In [20]:
tas_file = join(
    DATA_PATH_TEST_MARBLE_CMIP5_ROOT,
    "1pctCO2",
    "Amon",
    "tas",
    "CanESM2",
    "r1i1p1",
    "tas_Amon_CanESM2_1pctCO2_r1i1p1_185001-198912.nc",
)

In [22]:
tas_iris_load = SCMCube()
# you need this in case your cube has multiple variables
variable_constraint = iris.Constraint(
    cube_func=(lambda c: c.var_name == np.str("tas"))
)
# With iris v2.1.0, a bunch of 'Gracefully Filling' 
# warnings are thrown. This is a bug which will disappear 
# in iris v2.2.0 (see https://github.com/SciTools/iris/pull/3171)
# We filter them manually here
warnings.filterwarnings("ignore", ".*Gracefully filling.*")

tas_iris_load.cube = iris.load_cube(tas_file, variable_constraint)

/Users/zebedeenicholls/miniconda3/envs/netcdf-scm/lib/python3.6/site-packages/iris/fileformats/cf.py:798: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(message % (variable_name, nc_var_name))


The warning tells us that we need to add the areacella as a measure variable to our cube. Doing this manually everytime involves finding the areacella file, loading it, turning it into a cell measure and then adding it to the cube. This is a pain and involves about 100 lines of code. To make life easier, we wrap all of that away using `netcdf_scm`, which we will introduce in the next section.

### Loading with `netcdf_scm`

There are a couple of particularly annoying things involved with processing netCDF data. Firstly, the data is often stored in a folder hierarchy which can be fiddly to navigate. Secondly, the metadata is often stored separate from the variable cubes. 

Hence in `netcdf_scm`, we try to abstract the code which solves these two things away to make life a bit easier. This involves defining a cube in `netcdf_scm.iris_cube_wrappers`. The details can be read there, for now we just give an example.

Our example uses `MarbleCMIP5Cube`. This cube is designed to work with the CMIP5 data on our server at University of Melbourne, which has been organised into a number of folders which are similar, but not quite identical, to the CMOR directory structure described in section 3.1 of the [CMIP5 Data Reference Syntax](https://cmip.llnl.gov/cmip5/docs/cmip5_data_reference_syntax_v1-00_clean.pdf). To facilitate our example, the test data in `DATA_PATH_TEST_MARBLE_CMIP5_ROOT` is organised in the same way.

In any case, with this cube we can simply pass in the information about the data we want (experiment, model, ensemble member etc.) and it will load our desired cube. 

In [23]:
tas = MarbleCMIP5Cube()
tas.load_data(
    root_dir=DATA_PATH_TEST_MARBLE_CMIP5_ROOT,
    experiment="1pctCO2",
    modeling_realm="Amon",
    variable_name="tas",
    model="CanESM2",
    ensemble_member="r1i1p1",
)


NotImplementedError: 

We can verify that the loaded cube is exactly the same as the cube we loaded in the previous section (where we provided the full path).

In [24]:
assert tas.cube == tas_straight_load.cube

AttributeError: 'MarbleCMIP5Cube' object has no attribute 'cube'